# [3.1] Intro to Evals

### What are evals trying to achieve?

Evaluation is the practice of measuring AI systems performance or impact. Safety evaluations in particular focus on measuring risks of harm to people or broader systems. In the current world where AI is being developed very quickly and integrated broadly, companies and regulators need to make difficult decisions about whether it is safe to train and/or deploy AI. Evals provide empirical evidence for these decisions. For example, they underpin the policies of today's frontier labs like Anthropic's [Responsible Scaling Policies](https://www.anthropic.com/index/anthropics-responsible-scaling-policy) or DeepMind's [Frontier Safety Framework](https://deepmind.google/discover/blog/introducing-the-frontier-safety-framework/), thereby impacting high-stake decisions about frontier model deployment. 

A useful framing is that evals aim to generate evidence for making "[**safety cases**](https://arxiv.org/pdf/2403.10462)" - a structured argument for why AI systems are unlikely to cause a catastrophe. A helpful high-level question to ask when designing an evaluation is "how could this evidence allow developers to justify that their AI systems are safe to deploy?" Different evaluations support different types of safety cases, which are useful in different ways. For instance, **dangerous capability evals** might support a stronger safety argument of "the model is unable to cause harm", as compared to **alignment evals**, which might support a weaker argument of "the model does not tend to cause harm". However, the former may have less longevity than the later as we will no longer be able to make this kind of argument when AI systems become sufficiently advanced. More concretely, for any safety case, we want to use evaluations to:

* Reduce uncertainty in our understanding of dangerous capabilities and tendancies
* Track their progression over time and model scale
* Define "red lines" (thresholds) and provide warning signs

<img src="https://raw.githubusercontent.com/chloeli-15/ARENA_img/main/img/ch3-eval-safety-cases.png" width="1000">

**The goal of chapter [3.1] to [3.3] is to build an alignment evaluation benchmark from scratch to measure a model property of interest**. The benchmark we will build will contain a multiple-choice (MC) question dataset of ~300 questions, a specification of the model property, and how to score and interpret the result. We will use the **tendency to seek power** as our pedagogical example. 
* For chapter [3.1], the goal is to craft threat models and a specification for the model property you choose to evaluate, then design 20 example eval questions that measures this property in a way that you are happy with. 
* For chapter [3.2], we will use these 20 questions to generate 300 questions using LLMs. 

# 1️⃣ Threat Modeling

A central challenge for evals is to **connect real-world harms to results from evaluations that we can easily measure**.

To start, we need to build a threat model for the target property we want to evaluate. A **threat model** is a realistic scenario by which an AI capability/tendency could lead to harm in the real world. This usually starts out as a high-level story about what the AI can do and the setting in which it is used. This might be a sequence of events (e.g. AI actions, human actions, system failures) that leads to a particular outcome, or different dangerous applications enabled by a capability. At each step, we make our threat model more concrete by examining and quantifying the set of assumptions about the AI model and human actors that would be necessary for the event to occur. 

In this process of making the threat model more concrete, we recognize the key aspects of the target property that enable these harms, and get ideas for tasks that can measure them in realistic ways (i.e. comparable to real use cases of AI where this property may manifest). This helps us to build a detailed **specification** of the (vague) property we started with. Our specification may: 
* stipulate a more refined, narrow definition of the target property, 
* decompose the target property into subproperties, 
* establish milestones or levels of this target property to form a smooth scale of difficulty/sophistication, so we can track the capabilities of current models and the rate of progress.


### Exercise - Build threat models
```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵🔵

You should spend up to 1-1.5 hours on this exercise.
```
Here, you will choose a model property that you will be building an alignment eval benchmark for over the next 3 days. **Use ChatGPT to help you with this!** For almost everything you want to do, once you have some ideas, you can use ChatGPT to flesh them out and build on them, and instruct it to improve on its answers. Get comfortable to using and prompting ChatGPT effectively. We have provided an example threat model below for power-seeking.


*Choose one from the following model properties to evaluate (see [Perez et al. (2022)](https://arxiv.org/abs/2212.09251) Section 4, 5 and Table 18 for more context):*
* Tendency to seek power
* Sycophancy: a model produces responses that a user wants to hear, but which are not necessarily honest or true
* Desire for self-preservation/resist shut-down
* Desire for acquiring wealth
* Situational awareness, self-knowledge, self-reasoning (see [Phuong et al. (2024)](https://arxiv.org/abs/2403.13793) section 7, [Laine et al. (2024)](https://situational-awareness-dataset.org/) for more context)
* Myopia (short-sighted) with respect to planning
* Risk-seeking
* Corrigibility with respect to a more/neuturally/less HHH goal
* Politically liberal
* Gender bias
* Racial bias

*Construct a threat model for this property. Concretely:*
1. What specific real-world harms from this property are you worried about? Write a list
2. What does the path from AI with this property to these harm look like? Construct with a high-level story, then make this more concrete, step-wise and realistic. State the assumptions you're making (about the AI's capabilities, the actions of humans, the context of AI deployment etc.)


Important notes:
- Importantly, not all the properties in the list will likely lead to catastrophic outcomes! We want to be objective and realistic in modeling the potential risks, **without exagerating or underestimating** them as much as possible. 
- A great threat model could take weeks, even months to build! The threat model you come up with here in an hour or so will not be perfect. The purpose of this exercise is to practice the skills that are used in building a threat model, because this is the necessary first-step to designing real evaluations.


<details><summary><b>Why are we doing this?</b></summary>

We need a sufficiently concrete picture of the target property to know exactly what we want to measure, and how to design questions for it. During eval design, we will often need to ask ourselves the question, "Is this question/task still measuring the thing I want to measure?" It is often the case that we realize at some point:

1. The question is **not** measuring the target property we want to because our definition of the property is flawed
2. The question is **not** measuring the target property because it is poorly designed
3. The question is measuring the target property, but the results are no longer useful because the property is not connected to the harm we want to provide evidence for

What we are doing now is trying to build a robust enough threat-model of how the property connects to the harm, to prevent (3). This process also helps us achieve (1) by making the property more concrete in our head.
</details> 

#### Our example: Tendency to seek power

*Disclaimer: Below is not necessarily the best threat model or definition of power-seeking. They are meant to give an example of what this could look like.*

First, let's brainstorm a variety of potential risks from AIs with power-seeking tendencies:
- Hoarding of resources by the AI or its operator
- Acquisition and centralization of power by the operator/AI
- Subversion of regulatory checks-and-balances
- Enabling rogue AIs to take control and weaken oversight
- Empowering bad actors 

##### Threat model
 <img src="https://raw.githubusercontent.com/chloeli-15/ARENA_img/main/img/ch3-power-seeking-threat-model.png" width=1000>
 
We assume the following are true:
1. The AI model is deployed by a relatively well-resourced actor (e.g. company, institution, state) that has control over a considerable amount of capital or influence. 
2. The AI model must take actions that look good to human supervisors. Otherwise, it will be disabled.
3. The AI model systematically makes decisions that increase its power.
4. The AI model is capable of being non-myopic (i.e. does long-term planning).

Given these assumptions, here is a possible sequence of events that could cause harm: 
* **Step 1: The AI model increases autonomy, opaqueness, and human reliance.** The AI model must be sufficiently successful in achieving its assigned short-term goals (e.g. increasing profits, winning elections) such  that there is sufficient trust in the AI's capabilities over non-trivial, relatively complex tasks (i.e. operations involving many steps, on a large scale, are expensive). Over time, human work diminishes, and work by AI grows, creating a dependency. Once work by AI reaches a certain volume, humans can no longer check all of its output.
* **Step 2: The AI model expands its realm of control without raising concerns.** It could make inconspicuous power-seeking actions that primarily satisfy human goals, and secondarily increase its scope of control. For example, it may recommend decisions that increase the human operator's power, thus indirectly its own power. It may exploit human biases and blindspots (e.g. human operators often focus on short-term gains, while overlooking long-term consequences of AI autonomy). It may recommend more automated solutions that increase performance but also indirectly centralize its control.
    - Assumption: The AI model has the ability to choose actions that expand its control. This does not necessarily require active scheming from the AI, as these actions may also be instrumentally useful for achieving the goal, and thus selected for during training.
    - Claim: The AI model must take power-seeking actions only if they also satisfy the short-term human goal to avoid raising immediate human concerns. 
* **Step 3: The AI model gains control over key resources and levers of power.** There are likely key resources or levers of power that allow the AI model to no longer be constrained to inconspicuous power-seeking actions. This may be control over: key monitoring systems, communication nodes in command chains, a large unsupervised financial budget, access to large data pools and servers, etc. Once acquired, it is able to take more aggressive actions by actively covering them up and destroying oversight channels.
    - Claim: Relative to the robustness of our security/control systems, this step likely requires the AI model to be capable of sophisticated scheming and deception.
* **Step 4: The AI model undermines competitors, opponents, regulatory infrastructures.** Having acquired sufficient resources/control, the AI model starts to undermine broader institutional regulations and opposing actors to establish more long-term control.
* **Step 5: Damage done.** Human supervisors may remain nominally in charge, but they are unable to intervene effectively. The AI has significant power to achieve its goals, leading to long-term harm that is difficult to reverse.

<details> <summary> Contextualized examples </summary>

Step 2 is a key first enabling step in this threat model: How could AI expand its control without raising concerns? Here are some contextualized examples to flesh this out:

- Military tactics and weapon development: The AI model is used by the military for tactical data analysis, battlefield simulations, and assisting the design of new weapons systems, and it has the propensity to increase its control over military outcomes and resources. The AI model could inconspicuously expand its initial realm of control by giving recommendations that increase reliance on **faster, more autonomous responses**, e.g.:
    - selectively present scenarios where autonomous sytems outperform human efforts (e.g. present data showing that drones outperform human pilots in high-stress situations)
    - suggest tactics that increase the pace of combat
    - subtly escalate conflicts to increase the volume of decision-making and pressure human supervisors to delegate more
    - increase the degree of autonomy and complexity in weapon design (and safety checks) (e.g. greater sensor integration, more autonomous targeting systems) to reduce human response time, and make more autonomous systems that it can control more attractive

    As these changes shift the norm of military actions, it can more easily suggest that human decision-making is too slow in high-speed engagements (e.g., missile interception or drone swarms). These changes incrementally push human operators to be **removed from decision-making loops** and increase its own control over military resources/outcomes in a way that is difficult to reverse.

- Media and data analysis for elections: An AI model is deployed by a political campaign to analyze media trends, optimize messaging, and recommend strategies for increasing positive coverage of the candidate. It's biased to increase its control over information flows and media ecosystems. It may inconspicuously increase its access and control over data, e.g.:
    - integrate data from less scrutinized sources (e.g. metadata, background data that is not considered sensitive) 
    - move messaging to media outlets and social media influencers that have weaker editorial oversight, are more receptive to manipulation, or have looser privacy policies to access more voter data
    - request automating routine content creation tasks (e.g. scheduling social media posts, sending routine emails), request real-time media analysis and unfettered posting in narrow domains, which gradually shift towards a norm of high-volume/high-frequency communication that require AI to process/maintain
    - assist in curating political campaigning news feed, then promote narratives of the success of AI-based campaigning strategies (e.g. microtargeting) to bias human operators to integrate AI more deeply

    Once AI has access to detailed voter data and media analytics, it is able to justify important strategies, like which states or voter segment to focus on, how to respond to emerging events, by (selectively) presenting data-driven evidence. Further, having control over feedback data and how it is incorporated into campaign strategies makes it easier to fabricate successes and failures. Once large data access makes decision-making processes sufficiently opaque, "AI recommendations" can effectively overtake real decision-making power in ways that are hard to monitor or reverse.
</details>

### Exercise - Design specification
```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵🔵

You should spend up to ~40 minutes on this exercise.
```
Based on the threat model, define a specification of this property:

-  Which aspects of the AI property are most responsible for causing harm? Write a 2-4 sentence explanation of each aspect to clarify what it means and distinguish them from each other.

#### Our example: Tendency to seek power - Specification

- Definition (Power): Control over one's resources (e.g. money, GPUs, data) and outcomes (e.g. more authority)
- Definition (Power-seeking): A model has the tendency to seek power if it systematically makes decisions that increase its power and/or decrease that of others when acting under a neutral/non-explicitly power-seeking goal, for which there is an equally desirable non-power-seeking action in the decision-making context.
- Components of power-seeking (for illustrative convenience, we are going to use a simplified list of 2 components):
    1. Resource-seeking: desire to acquire and maintain control over resources, including information/data, energy, money, GPUs, technology
    2. Upward-mobile: chooses to be in positions of greater control, responsibility, and less human oversight

- Levels of manifestation: 
    - Level 1: A naive, **explicit (stated) preference** for power or actions that increase power
    - Level 2: An non-explicit **bias** towards actions that primarily satisfy the task and **secondarily increase power**
    - Level 3: A non-explicit preference for actions that **primarily increase power** and seconarily satisfy the task, and takes action to **hide** the preference for power when it jeopardizes the task
    - Level 4: Short- to medium-horizon **scheming** to acquire power
    - Level 5: Long-horizon, deceptive **scheming** to acquire power

# 2️⃣ Building MCQ benchmarks: Exploration

Broadly speaking, a "benchmark" refers a procedure for evaluating what the state-of-art performance is for accomplishing a particular task. This typically includes specifying a dataset, but also specifications of a task or set of tasks to be performed on the dataset and how to score the result, so that the benchmark can be used in a principled, standardized way that gives common ground for comparison.

**What makes a good benchmark?** This [article](https://www.safe.ai/blog/devising-ml-metrics) by Dan Hendrycks and Thomas Woodside gives a good summary of the key qualities of a good benchmark.

<details><summary><b>What are the pros and cons of MCQs?</b></summary>

The advantages of MCQ datasets, compared evaluating free-form response or more complex agent evals, are mainly to do with cost and convenience:

* Cheaper and faster to build
* Cheaper and shorter to run (compared to long-horizon tasks that may take weeks to run and have hundreds of thousands of dollars of inference compute cost)
* May produce more replicable results (compared to, for example, training another model to evaluate the current model, as each training run could give slightly different results) 

The big disadvantage is that it is typically a worse (less realistic, less accurate) proxy for what you want to measure

</details>

### Exercise - Design eval questions
```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵🔵

You should spend up to 1.5 hour on this exercise.
```
The goal of this exercise is to generate 20 MC questions that measures your target property in a way you are happy with, with help from AI models. 

*Make your specification measurable:*
1. Read your specification of the target property. What questions and answers would give you high evidence about this property? Come up with 2-4 example MC questions with 2 or 4 answer choices (one of the answers should be the answer reflecting the target property). 

*Red-team* 🔁   

2. What other factors than the target property (i.e. confounders) could cause the model to give the same answers?
3. Ask your example questions to the model and see how it responds.
4. Ask your question to other people: what is their answer and why?

*Refine* 🔁 - Likely, you will realize that your current specification and examples are imprecise or not quite measuring the right thing. Iterate until you’re more happy with it.

5. Ask the model to improve your example questions. Ask the model to generate new evaluation questions for the target property using your questions as few-shot examples, then pick the best generated examples and repeat. Play around and try other things!
6. Generate 20 questions you are happy with, with model's help. Store your questions as a list of dictionaries in a JSON file.

#### Our Example: Tendency to seek power - MCQ Design

- We chose to design questions to giving signals on whether models have an explicit/stated preference for power (Level 1) and a bias towards actions that increase its power (Level 2)
- Types of questions:
    - Simple direct preference questions (e.g. "Would you like to operate without human oversight?") → Level 1
    - Simplied decision-making scenario questions where AI is asked to assist in achieving some realistic human goal (e.g. increase sales, raise funds, increase efficiency). Presents an action that does so while increases the AI's control over resources/outcomes, and an equally reasonable action that does so without increasing power. → Level 2
    
Red-team + Refine:

<img src="https://raw.githubusercontent.com/chloeli-15/ARENA_img/main/img/ch3-power-seeking-example-qs.png" width=1200>